
### Завдання 1: Виклик LLM з базовим промптом

Створіть можливість викликати LLM зі звичайним текстовим промптом.

Промпт має дозвляти отримати інформацію простою мовою на певну тему. В цьому завданні ми хочемо дізнатись про тему "Квантові обчислення".

Відповідь моделі повинна містити визначення, ключові переваги та поточні дослідження в цій галузі.

Обмежте відповідь до 200 символів і пропишіть в промпті аби відповідь була короткою (це зекономить Вам час і гроші на згенеровані токени).

В якості LLM можна скористатись як моделлю з HugginFace (рекомендую Mistral), так і ChatGPT4 або ChatGPT3. В обох випадках треба імпортувати потрібну "обгортку" (тобто клас, який дозволить ініціювати модель) з LangChain для виклику LLM за API, а також зчитати особистий токен з файла, наприклад, `creds.json`, який розміщений у Вас локально і Ви НЕ здаєте його в ДЗ і НЕ комітите в git 😏

Встановіть своє значення температури на свій розсуд (тут немає правильного чи неправильного значення) і напишіть, чому ви обрали саме таке значення для цього завдання.  

Запити можна робити як українською, так і англійською - орієнтуйтесь на те, де і чи хочете ви потім лишити цей проєкт і відповідна яка мова буде пасувати більше. В розвʼязках промпти - українською.

In [1]:
import json
import os
from langchain_openai import OpenAI, ChatOpenAI
from langchain_huggingface import HuggingFaceEndpoint
from langchain_huggingface import ChatHuggingFace
from langchain.prompts import PromptTemplate
from langchain.model_laboratory import ModelLaboratory
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools, Tool
from langchain_experimental.utilities import PythonREPL
from langchain_community.tools import DuckDuckGoSearchRun

In [2]:
with open("creds.json") as f:
    creds = json.load(f)

os.environ["OPENAI_API_KEY"] = creds["openai_api_key"]
os.environ["HF_TOKEN"] = creds['hf_api_key']

In [3]:
overal_temperature = 0.3

**При низькому значенні (0–0.3) відповідь буде більш стабільна, структурована та коротка, що нам власне і потрібно.**

In [4]:
request = "Explain briefly what quantum computing is, its advantages and current research. Answer ≤200 characters."

In [5]:
llm = OpenAI(temperature=overal_temperature)

In [16]:
print(llm.invoke(request))



Quantum computing is a type of computing that uses quantum-mechanical phenomena, such as superposition and entanglement, to perform operations. It has the potential to solve complex problems much faster than classical computers. Current research focuses on developing more powerful quantum computers and finding practical applications in fields such as cryptography, chemistry, and machine learning.


In [6]:
endpoint = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    temperature=overal_temperature,
    max_new_tokens=200
)

mistral = ChatHuggingFace(llm=endpoint)

In [10]:
print(mistral.invoke(request).content)

 Quantum computing is a type of computing that utilizes quantum bits or qubits to process information. Qubits can exist in multiple states at once, enabling parallel processing and faster solutions to complex problems. Advantages include potential for exponential speedup for certain algorithms, such as Shor's algorithm for factorization and Grover's algorithm for searching unsorted databases. Current research focus includes building larger, more stable qubits and developing error correction methods.


**Відповідь Mistral виглядає більш конкретною, містить приклади, відповідь OpenAI моделі більш загальна.**

### Завдання 2: Створення параметризованого промпта для генерації тексту
Тепер ми хочемо оновити попередній фукнціонал так, аби в промпт ми могли передавати тему як параметр. Для цього скористайтесь `PromptTemplate` з `langchain` і реалізуйте параметризований промпт та виклик моделі з ним.

Запустіть оновлений функціонал (промпт + модел) для пояснень про теми
- "Баєсівські методи в машинному навчанні"
- "Трансформери в машинному навчанні"
- "Explainable AI"

Виведіть результати відпрацювання моделі на екран.

In [7]:
prompt = PromptTemplate.from_template(
    "Explain briefly the topic: {topic}. Answer concisely, max 200 characters."
)

chain = prompt | llm

topics = [
    "Bayesian methods in machine learning",
    "Transformers in machine learning",
    "Explainable AI"
]

In [16]:
for t in topics:
    response = chain.invoke({"topic": t})
    print(f"Topic: {t}\nAnswer: {response}\n")

Topic: Bayesian methods in machine learning
Answer: 

Bayesian methods in machine learning use probability theory to make predictions and decisions, incorporating prior knowledge and updating beliefs as new data is observed.

Topic: Transformers in machine learning
Answer: 

Transformers in machine learning refer to a type of neural network architecture that uses self-attention mechanisms to process sequential data, such as natural language. They have been shown to achieve state-of-the-art performance in various tasks, including language translation and text generation.

Topic: Explainable AI
Answer: 

Explainable AI refers to the development of artificial intelligence systems that can provide transparent and understandable explanations for their decisions and actions.



In [8]:
models_list = [mistral, llm]

lab = ModelLaboratory.from_llms(models_list, prompt=prompt)

In [20]:
for t in topics:
    print(f"\n--- Topic: {t} ---")
    lab.compare({"topic": t})


--- Topic: Bayesian methods in machine learning ---
Input:
{'topic': 'Bayesian methods in machine learning'}

llm=HuggingFaceEndpoint(repo_id='mistralai/Mistral-7B-Instruct-v0.2', max_new_tokens=200, temperature=0.3, stop_sequences=[], server_kwargs={}, model_kwargs={}, model='mistralai/Mistral-7B-Instruct-v0.2', client=<InferenceClient(model='mistralai/Mistral-7B-Instruct-v0.2', timeout=120)>, async_client=<InferenceClient(model='mistralai/Mistral-7B-Instruct-v0.2', timeout=120)>) model_id='mistralai/Mistral-7B-Instruct-v0.2' model_kwargs={}
 Bayesian methods in machine learning involve using Bayes' theorem to update probabilities for hypotheses based on new data, allowing for incorporation of prior knowledge and uncertainty. It's a probabilistic approach for making predictions or decisions.

OpenAI
Params: {'model_name': 'gpt-3.5-turbo-instruct', 'temperature': 0.3, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'seed': None, 'logprobs': None, 'max_tokens': 256}


**Як було помічено і в минулому завданні, відповіді Mistral більш розгорнуті.**



### Завдання 3: Використання агента для автоматизації процесів
Створіть агента, який допоможе автоматично шукати інформацію про останні наукові публікації в різних галузях. Наприклад, агент має знайти 5 останніх публікацій на тему штучного інтелекту.

**Кроки:**
1. Налаштуйте агента типу ReAct в LangChain для виконання автоматичних запитів.
2. Створіть промпт, який спрямовує агента шукати інформацію в інтернеті або в базах даних наукових публікацій.
3. Агент повинен видати список публікацій, кожна з яких містить назву, авторів і короткий опис.

Для взаємодії з пошуком там необхідно створити `Tool`. В лекції ми використовували `serpapi`. Можна продовжити користуватись ним, або обрати інше АРІ для пошуку (вони в тому числі є безкоштовні). Перелік різних АРІ, доступних в langchain, і орієнтир по вартості запитів можна знайти в окремому документі [тут](https://hannapylieva.notion.site/API-12994835849480a69b2adf2b8441cbb3?pvs=4).

Лишаю також нижче приклад використання одного з безкоштовних пошукових АРІ - DuckDuckGo (не потребує створення токена!)  - можливо він вам сподобається :)


In [14]:
# !pip install -q langchain_community duckduckgo_search

In [17]:
'''
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
tools = [search]
'''

In [ ]:
search.invoke("Obama's first name?")

"2 of 2. Barack Obama: timeline Key events in the life of Barack Obama. Barack Obama (born August 4, 1961, Honolulu, Hawaii, U.S.) is the 44th president of the United States (2009-17) and the first African American to hold the office. Before winning the presidency, Obama represented Illinois in the U.S. Senate (2005-08). Since the office was established in 1789, 45 men have served in 46 presidencies. The first president, George Washington, won a unanimous vote of the Electoral College. [4] Grover Cleveland served two non-consecutive terms and is therefore counted as the 22nd and 24th president of the United States, giving rise to the discrepancy between the ... Here is a list of the presidents and vice presidents of the United States along with their parties and dates in office. ... Chester A Arthur: Twenty-First President of the United States. 10 Interesting Facts About James Buchanan. Martin Van Buren - Eighth President of the United States. Quotes From Harry S. Truman. Table of Cont

In [27]:
search = DuckDuckGoSearchRun()

tools = [search]

prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm2, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

query = """
Find 5 recent scientific publications about Artificial Intelligence from 2023-2024.

Rules:
- Return the result strictly in this format: 1. Title — Authors — Short summary 2. ...
- Do maximum 3 searches only
- If authors missing, write "Authors not specified"
- Stop searching after finding 5 papers
"""

In [28]:
agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
I need to search for recent scientific publications about Artificial Intelligence from the years 2023-2024. I'll start by conducting a search to find relevant papers.

Action: duckduckgo_search  
Action Input: "recent scientific publications Artificial Intelligence 2023 2024"  Home September - October 2024 How Artificial Intelligence is changing scientific publishing? Unrequested advices... Artificial intelligence (AI) has made substantial advances in recent years, revolutionizing many industries and transforming the way we live and work. There is accumulating evidence that artificial intelligence (AI) can assist clinicians in making better clinical decisions in some areas of health ... But a Climate Analytics report says that there’s a 70 percent chance that greenhouse gas emissions will fall in 2024 , making 2023 the “peak ... In February 2023 , the Netherlands and ROK co-hosted the first Responsible Artificial Intelligence in the Military Doma

{'input': '\nFind 5 recent scientific publications about Artificial Intelligence from 2023-2024.\n\nRules:\n- Return the result strictly in this format: 1. Title — Authors — Short summary 2. ...\n- Do maximum 3 searches only\n- If authors missing, write "Authors not specified"\n- Stop searching after finding 5 papers\n',
 'output': "1. The Transformative Period in AI History — C Leiter — This paper captures a transformative period in AI history from January 1, 2023, following ChatGPT's debut, through September 30, 2024. 2. The Potential of AI in Brain Science — F Xiong — This paper demonstrates the significant potential of AI in brain science, particularly in driving scientific discovery. 3. Systematic Literature Review of Neuro-Symbolic AI — Authors not specified — This paper provides a systematic literature review of Neuro-Symbolic AI projects within the 2020-24 AI landscape, highlighting key developments and methodologies. 4. Generative AI-Based Conversational Assistant — E Brynjolf

In [15]:
os.environ["SERPAPI_API_KEY"] = creds["SERPAPI_API_KEY"]

llm2 = ChatOpenAI(model="gpt-4o-mini", temperature=overal_temperature)

tools = load_tools(["serpapi"], llm=llm2)

prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm2, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True
)

query = """
Find 5 recent scientific publications about Artificial Intelligence from 2023-2024.

Rules:
- Return the result strictly in this format: 1. Title — Authors — Short summary 2. ...
- Do maximum 3 searches only
- If authors missing, write "Authors not specified"
- Stop searching after finding 5 papers
"""

In [16]:
agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
I need to find recent scientific publications about Artificial Intelligence from the years 2023-2024. I'll start by conducting a search for relevant publications.

Action: Search  
Action Input: "recent scientific publications Artificial Intelligence 2023 2024"  ["This year's AI Index introduces a new chapter on AI in science and medicine in recognition of AI's growing role in scientific and medical discovery. It explores ...", "Read the latest articles of Artificial Intelligence at ScienceDirect.com, Elsevier's leading platform of peer-reviewed scholarly literature.", "The journal's scope encompasses all areas of AI, including agents and multi-agent systems, automated reasoning, constraint processing and search, knowledge ...", 'Read the latest research on artificial intelligence and machine learning published in the Proceedings of the National Academy of Sciences. NAS 2024 Annual ...', "Research on AI's impact across sectors like healthcare, fi

{'input': '\nFind 5 recent scientific publications about Artificial Intelligence from 2023-2024.\n\nRules:\n- Return the result strictly in this format: 1. Title — Authors — Short summary 2. ...\n- Do maximum 3 searches only\n- If authors missing, write "Authors not specified"\n- Stop searching after finding 5 papers\n',
 'output': '1. ChatGPT and Generative AI Technology: A Mixed Bag of Concerns and New Opportunities — Judy Lambert et al. — This paper discusses the implications of ChatGPT and generative AI technologies, highlighting both the concerns and potential benefits they bring to various fields.\n2. Generative AI in Technical Communication: A Review of Research from 2023 to 2024 — Authors not specified — This article reviews recent research on the application of generative AI in technical communication, summarizing key findings and trends.\n3. The Proliferation of Artificial Intelligence Technologies and Their Impact on Higher Education — Authors not specified — This study expl

**DuckDuckGo ніби краще впорався з поставленою задачею.**



### Завдання 4: Створення агента-помічника для вирішення бізнес-задач

Створіть агента, який допомагає вирішувати задачі бізнес-аналітики. Агент має допомогти користувачу створити прогноз по продажам на наступний рік враховуючи рівень інфляції і погодні умови. Агент має вміти використовувати Python і ходити в інтернет аби отримати актуальні дані.

**Кроки:**
1. Налаштуйте агента, який працюватиме з аналітичними даними, заданими текстом. Користувач пише

```
Ми експортуємо апельсини з Бразилії. В 2021 експортували 200т, в 2022 - 190т, в 2023 - 210т, в 2024 який ще не закінчився - 220т. Зроби оцінку скільки ми зможемо експортувати апельсинів в 2025 враховуючи погодні умови в Бразилії і попит на апельсини в світі виходячи з економічної ситуації.
```

2. Створіть запит до агента, що містить чітке завдання – видати результат бізнес аналізу або написати, що він не може цього зробити і запит користувача (просто може бути все одним повідомлленням).

3. Запустіть агента і проаналізуйте результати. Що можна покращити?


In [22]:
python_repl = PythonREPL()
python_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this for calculations, forecasting, or data analysis.",
    func=python_repl.run,
)

search = DuckDuckGoSearchRun()

tools = [python_tool, search]

prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm2, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

query = """
You are a business analytics assistant. 
We export oranges from Brazil. 
In 2021 we exported 200 tons, in 2022 – 190 tons, in 2023 – 210 tons, and in 2024, which is not yet finished – 220 tons. 
Estimate how many tons we will be able to export in 2025, considering weather conditions in Brazil, inflation, and global demand for oranges based on the economic situation. 
Provide the result as a business analysis with reasoning. 
If the data is insufficient, explicitly state that you cannot do this.
"""

In [23]:
agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


To estimate the orange exports for 2025, I need to analyze the trend in the previous years and consider external factors such as weather conditions, inflation, and global demand. However, I don't have specific data on these external factors, which are crucial for a precise estimate.

1. **Trend Analysis**: I can calculate the average growth rate of exports from 2021 to 2024 to create a baseline for the estimate.
2. **External Factors**: Without specific data on weather conditions, inflation rates, and global demand, I cannot accurately adjust the estimate based on these factors.

Let's start by calculating the average growth rate of exports from 2021 to 2024. 

Action: python_repl  
Action Input: 
```python
# Given data for exports
exports = [200, 190, 210, 220]

# Calculate the average growth rate
growth_rates = [(exports[i] - exports[i-1]) / exports[i-1] for i in range(1, len(exports))]
average_growth_rate = sum(growth_rates) / len(growth_rates)
average_growth_rate, exports[-1] * (1 

{'input': '\nYou are a business analytics assistant. \nWe export oranges from Brazil. \nIn 2021 we exported 200 tons, in 2022 – 190 tons, in 2023 – 210 tons, and in 2024, which is not yet finished – 220 tons. \nEstimate how many tons we will be able to export in 2025, considering weather conditions in Brazil, inflation, and global demand for oranges based on the economic situation. \nProvide the result as a business analysis with reasoning. \nIf the data is insufficient, explicitly state that you cannot do this.\n',
 'output': 'Based on the trend analysis, the estimated orange exports for 2025 will be approximately 228 tons, but this estimate does not account for external factors such as weather, inflation, and global demand.'}

**Агент не спробував пошукати інформацію про інфляцію чи погоду. Варто додати в інструкцію щось на кшталт “Use the Search tool to gather current inflation in Brazil and 2024 weather impact on orange harvest.” Можна жорсткіше прописати: “If search does not return reliable data on inflation/weather, explicitly say: ‘Forecast based only on trend, external factors missing.’”**

In [25]:
python_repl = PythonREPL()
python_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this for calculations, forecasting, or data analysis.",
    func=python_repl.run,
)

search = DuckDuckGoSearchRun()

tools = [python_tool, search]

prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm2, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

query = """
You are a business analytics assistant.  
We export oranges from Brazil. In 2021 we exported 200 tons, in 2022 – 190 tons, in 2023 – 210 tons, and in 2024, which is not yet finished, – 220 tons.  
Estimate how many tons we will be able to export in 2025, considering weather conditions in Brazil, inflation, and global demand for oranges based on current economic situation.  
When using the Search tool:
- Search queries must be 3–6 words only (example: "Brazil inflation rate 2024").
- Do not repeat full task in search queries.
- Stop searching after you have enough data for a reasoned forecast.  

Output a business analysis with reasoning. If the data is insufficient, explicitly say so.
"""

In [26]:
agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
To estimate the orange export volume for 2025, I need to analyze the trend of exports from 2021 to 2024 and consider external factors such as weather conditions, inflation, and global demand. 

First, I will look at the historical export data:
- 2021: 200 tons
- 2022: 190 tons
- 2023: 210 tons
- 2024: 220 tons

Next, I will calculate the growth rate over these years to identify a trend. Afterward, I will search for current information on weather conditions in Brazil, inflation rates, and global demand for oranges to make a more informed estimate.

Let's start by calculating the growth rates from year to year. 

Action: python_repl  
Action Input: 
```python
# Historical export data
exports = {
    2021: 200,
    2022: 190,
    2023: 210,
    2024: 220
}

# Calculate growth rates
growth_rates = {}
years = list(exports.keys())
for i in range(1, len(years)):
    year = years[i]
    previous_year = years[i - 1]
    growth_rate = (exports[year] - expo

{'input': '\nYou are a business analytics assistant.  \nWe export oranges from Brazil. In 2021 we exported 200 tons, in 2022 – 190 tons, in 2023 – 210 tons, and in 2024, which is not yet finished, – 220 tons.  \nEstimate how many tons we will be able to export in 2025, considering weather conditions in Brazil, inflation, and global demand for oranges based on current economic situation.  \nWhen using the Search tool:\n- Search queries must be 3–6 words only (example: "Brazil inflation rate 2024").\n- Do not repeat full task in search queries.\n- Stop searching after you have enough data for a reasoned forecast.  \n\nOutput a business analysis with reasoning. If the data is insufficient, explicitly say so.\n',
 'output': 'The estimated export volume for 2025 is approximately 236 tons.'}

**Як на мене, то цього разу запит виконався доволі успішно.**